#Load Data

In [ ]:
!pip install matplotlib scipy opencv-python pandas

In [ ]:
!apt-get update
!apt-get install unzip
!apt-get install ffmpeg libsm6 libxext6  -y

In [ ]:
!wget 'https://www.dropbox.com/s/l0y7cqqymjef2sh/tiny-imagenet-200.zip?dl=0'
!mv 'tiny-imagenet-200.zip?dl=0' 'ImageNetData.zip'
!unzip 'ImageNetData.zip'

In [4]:
!ls

ImageNet.ipynb	  imagenet  onstart.log  ports.log
ImageNetData.zip  models    onstart.sh	 utils.py


In [5]:
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [6]:
class ImageNet(Dataset):
    def __init__(self):
        self.imgs_path = "imagenet/"
        sub_folders = glob.glob(self.imgs_path + "*")
        self.data = []
        self.classes = []
        for sub_folder in sub_folders:
            file_list = glob.glob(sub_folder + "/*")
            for class_path in file_list:
                class_name = class_path.split("/")[-1]
                for img_path in glob.glob(class_path + "/*.JPEG"):
                    self.data.append([img_path, class_name])
                    self.classes.append(class_name)
        self.classes = list(set(self.classes))
        self.class_map = dict()
        for i in range(len(self.classes)):
            self.class_map[self.classes[i]] = i

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img).float()
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor(class_id)
        return img_tensor, class_id

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision

from torchvision import *
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import time
import copy
import os

from models import *
from utils import progress_bar

batch_size = 32

transforms = transforms.Compose(
[
    transforms.ToTensor()
])

dataset = ImageNet()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

images, labels = next(iter(train_dataloader)) 
print("images-size:", images.shape)

out = torchvision.utils.make_grid(images)
print("out-size:", labels.shape)

images-size: torch.Size([32, 3, 64, 64])
out-size: torch.Size([32])


In [12]:
class EarlyStopping():
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if train_loss < self.min_delta:
            self.min_delta -= self.min_delta*0.5
            
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True
        else:
            self.counter = 0

# Training

In [13]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(train_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return train_loss


# Testing

In [14]:
def test(epoch,model_name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(test_dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/'+model_name+'.pth')
        best_acc = acc
    return test_loss


In [19]:
classes = dataset.classes
lr = 0.01
n_epochs = 20
in_dim = 64

In [17]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = densenet_cifar(in_dim=in_dim,num_classes=len(classes))
net = net.to(device)
if device == 'cuda:0':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DenseNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0


KeyboardInterrupt: 

In [20]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = PreActResNet18(in_dim=in_dim,num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'PreActResNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1



Epoch: 0
 [================================================================>]  Step: 39ms | Tot: 1m51s | Loss: 4.251 | Acc: 11.193% (9850/8800 2750/2750   =>...........................................................]  Step: 41ms | Tot: 9s166ms | Loss: 5.239 | Acc: 2.142% (157/732 229/2750 =======>.........................................................]  Step: 40ms | Tot: 13s320ms | Loss: 5.140 | Acc: 2.672% (283/1059 331/2750 ==========>......................................................]  Step: 41ms | Tot: 17s388ms | Loss: 5.069 | Acc: 2.973% (410/1379 431/2750 ==========>......................................................]  Step: 41ms | Tot: 17s794ms | Loss: 5.062 | Acc: 3.005% (424/1411 441/2750 ===========>.....................................................]  Step: 41ms | Tot: 19s891ms | Loss: 5.031 | Acc: 3.264% (515/1577 493/2750 ============>....................................................]  Step: 42ms | Tot: 21s141ms | Loss: 5.012 | Acc: 3.375% (566/1676 524/275

 [================================================================>]  Step: 40ms | Tot: 1m51s | Loss: 2.943 | Acc: 31.556% (27769/8800 2750/2750 0  ==========>....................................................]  Step: 40ms | Tot: 20s714ms | Loss: 3.022 | Acc: 29.958% (4937/1648 515/2750 ================>................................................]  Step: 40ms | Tot: 28s542ms | Loss: 3.014 | Acc: 30.160% (6833/2265 708/2750 =================>...............................................]  Step: 41ms | Tot: 30s190ms | Loss: 3.018 | Acc: 30.114% (7208/2393 748/2750 =================>...............................................]  Step: 40ms | Tot: 30s683ms | Loss: 3.015 | Acc: 30.148% (7332/2432 760/2750 =================>...............................................]  Step: 40ms | Tot: 30s764ms | Loss: 3.015 | Acc: 30.147% (7351/2438 762/2750 ==================>..............................................]  Step: 41ms | Tot: 30s966ms | Loss: 3.015 | Acc: 30.154% (7401/2454

 [================================================================>]  Step: 39ms | Tot: 1m50s | Loss: 2.408 | Acc: 42.017% (36975/8800 2750/2750 0  ........................................................]  Step: 40ms | Tot: 923ms | Loss: 2.439 | Acc: 41.193% (290/70 22/2750 >................................................................]  Step: 40ms | Tot: 1s652ms | Loss: 2.375 | Acc: 42.109% (539/128 40/2750 ==========>......................................................]  Step: 41ms | Tot: 17s913ms | Loss: 2.392 | Acc: 42.148% (5894/1398 437/2750 ===========>.....................................................]  Step: 41ms | Tot: 19s937ms | Loss: 2.388 | Acc: 42.037% (6551/1558 487/2750 ===============>.................................................]  Step: 40ms | Tot: 26s509ms | Loss: 2.405 | Acc: 41.745% (8683/2080 650/2750 =================>...............................................]  Step: 40ms | Tot: 30s369ms | Loss: 2.402 | Acc: 41.907% (10004/2387 746/2750 =======

 [================================================================>]  Step: 40ms | Tot: 1m50s | Loss: 2.018 | Acc: 49.667% (43707/8800 2750/2750 0  ...........................................................]  Step: 41ms | Tot: 1s427ms | Loss: 1.929 | Acc: 51.964% (582/112 35/2750 ======>..........................................................]  Step: 40ms | Tot: 10s571ms | Loss: 1.898 | Acc: 51.961% (4373/841 263/2750 ==========>......................................................]  Step: 41ms | Tot: 17s327ms | Loss: 1.946 | Acc: 51.204% (7062/1379 431/2750 ==========>......................................................]  Step: 41ms | Tot: 18s13ms | Loss: 1.946 | Acc: 51.290% (7353/1433 448/2750 ============>....................................................]  Step: 41ms | Tot: 20s678ms | Loss: 1.942 | Acc: 51.246% (8429/1644 514/2750 =================>...............................................]  Step: 40ms | Tot: 30s84ms | Loss: 1.949 | Acc: 51.067% (12207/2390 747/2750 

 [================================================================>]  Step: 39ms | Tot: 1m50s | Loss: 1.704 | Acc: 56.543% (49758/8800 2750/2750 0  .............................................................]  Step: 41ms | Tot: 6s654ms | Loss: 1.504 | Acc: 61.672% (3276/531 166/2750 =====>...........................................................]  Step: 40ms | Tot: 9s518ms | Loss: 1.529 | Acc: 60.970% (4624/758 237/2750 ======>..........................................................]  Step: 40ms | Tot: 11s448ms | Loss: 1.529 | Acc: 60.669% (5533/912 285/2750 =========>.......................................................]  Step: 40ms | Tot: 16s207ms | Loss: 1.534 | Acc: 60.623% (7818/1289 403/2750 ==========>......................................................]  Step: 41ms | Tot: 17s780ms | Loss: 1.541 | Acc: 60.379% (8540/1414 442/2750 ==========>......................................................]  Step: 40ms | Tot: 17s983ms | Loss: 1.542 | Acc: 60.368% (8635/1430 447/27

 [================================================================>]  Step: 40ms | Tot: 1m50s | Loss: 1.412 | Acc: 62.358% (54875/8800 2750/2750 0  ............................................................]  Step: 40ms | Tot: 2s223ms | Loss: 1.101 | Acc: 70.982% (1272/179 56/2750 =========>.......................................................]  Step: 41ms | Tot: 16s329ms | Loss: 1.149 | Acc: 69.171% (9031/1305 408/2750 =============>...................................................]  Step: 40ms | Tot: 22s271ms | Loss: 1.184 | Acc: 68.194% (12133/1779 556/2750 ==============>..................................................]  Step: 41ms | Tot: 25s89ms | Loss: 1.192 | Acc: 67.831% (13588/2003 626/2750 ===============>.................................................]  Step: 41ms | Tot: 25s774ms | Loss: 1.197 | Acc: 67.695% (13929/2057 643/2750 =================>...............................................]  Step: 40ms | Tot: 30s475ms | Loss: 1.224 | Acc: 66.974% (16288/2432 76

 [================================================================>]  Step: 41ms | Tot: 1m49s | Loss: 1.263 | Acc: 65.802% (57906/8800 2750/2750 0  =========>......................................................]  Step: 40ms | Tot: 17s20ms | Loss: 0.970 | Acc: 73.608% (10364/1408 440/2750 =================>...............................................]  Step: 40ms | Tot: 28s633ms | Loss: 1.033 | Acc: 71.631% (16710/2332 729/2750 ==================>..............................................]  Step: 40ms | Tot: 30s840ms | Loss: 1.045 | Acc: 71.361% (17903/2508 784/2750 ===========================>.....................................]  Step: 40ms | Tot: 46s225ms | Loss: 1.118 | Acc: 69.548% (25972/3734 1167/2750 =============================>...................................]  Step: 41ms | Tot: 48s841ms | Loss: 1.129 | Acc: 69.257% (27304/3942 1232/2750 =============================>...................................]  Step: 41ms | Tot: 49s245ms | Loss: 1.131 | Acc: 69.188% (27

 [================================================================>]  Step: 41ms | Tot: 1m50s | Loss: 1.020 | Acc: 71.374% (62809/8800 2750/2750 0  =>...........................................................]  Step: 41ms | Tot: 9s297ms | Loss: 0.677 | Acc: 81.397% (6095/748 234/2750 ======>..........................................................]  Step: 39ms | Tot: 11s110ms | Loss: 0.680 | Acc: 81.149% (7245/892 279/2750 =======>.........................................................]  Step: 41ms | Tot: 12s158ms | Loss: 0.682 | Acc: 81.045% (7910/976 305/2750 =========>.......................................................]  Step: 41ms | Tot: 16s329ms | Loss: 0.694 | Acc: 80.723% (10565/1308 409/2750 ==========>......................................................]  Step: 41ms | Tot: 17s419ms | Loss: 0.705 | Acc: 80.297% (11203/1395 436/2750 ===========>.....................................................]  Step: 42ms | Tot: 19s506ms | Loss: 0.718 | Acc: 79.921% (12506/1564 48

In [ ]:
del net

In [21]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DPN92(in_dim=in_dim,num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DPN') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 274ms | Tot: 12m34s | Loss: 5.087 | Acc: 2.772% (2439/8800 2750/2750 0 
 [================================================================>]  Step: 49ms | Tot: 44s348ms | Loss: 4.811 | Acc: 7.018% (1544/2200 688/688 =======================>.......................................]  Step: 67ms | Tot: 17s588ms | Loss: 4.852 | Acc: 7.200% (629/873 273/688 
Saving..

Epoch: 1
 [================================================================>]  Step: 275ms | Tot: 12m34s | Loss: 4.082 | Acc: 12.791% (11256/8800 2750/2750 0 
 [================================================================>]  Step: 37ms | Tot: 44s384ms | Loss: 5.005 | Acc: 18.677% (4109/2200 688/688 
Saving..

Epoch: 2
 [================================================================>]  Step: 275ms | Tot: 12m34s | Loss: 3.358 | Acc: 23.956% (21081/8800 2750/2750 0 
 [================================================================>]  Step:

In [ ]:
del net

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = EfficientNetB0(num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'EffecientNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1


Epoch: 0
 [================================================================>]  Step: 77ms | Tot: 3m21s | Loss: 4.379 | Acc: 8.510% (7489/8800 2750/2750 0 =========================================================>.......]  Step: 67ms | Tot: 3m345ms | Loss: 4.433 | Acc: 7.864% (6173/7849 2453/2750 ===========================================================>.....]  Step: 67ms | Tot: 3m3s | Loss: 4.424 | Acc: 7.993% (6405/8012 2504/2750 
 [================================================================>]  Step: 29ms | Tot: 17s441ms | Loss: 3.884 | Acc: 14.214% (3127/2200 688/688 ================>.............................................]  Step: 25ms | Tot: 5s336ms | Loss: 3.901 | Acc: 13.863% (936/675 211/688 ==========================>......................................]  Step: 25ms | Tot: 7s108ms | Loss: 3.890 | Acc: 14.074% (1261/896 280/688 ==========================>......................................]  Step: 26ms | Tot: 7s134ms | Loss: 3.889 | Acc: 14.057% (1264/899 281/6

 [================================================================>]  Step: 71ms | Tot: 3m13s | Loss: 2.641 | Acc: 36.685% (32283/8800 2750/2750 0 =====>.........................................................]  Step: 67ms | Tot: 21s903ms | Loss: 2.570 | Acc: 37.874% (3745/988 309/2750 ============>....................................................]  Step: 67ms | Tot: 37s137ms | Loss: 2.588 | Acc: 37.524% (6280/1673 523/2750 ============================>....................................]  Step: 66ms | Tot: 1m27s | Loss: 2.614 | Acc: 37.152% (14540/3913 1223/2750 =========================================================>.......]  Step: 65ms | Tot: 2m51s | Loss: 2.635 | Acc: 36.768% (28591/7776 2430/2750 
 [================================================================>]  Step: 22ms | Tot: 17s558ms | Loss: 2.716 | Acc: 35.709% (7856/2200 688/688 
Saving..

Epoch: 7
 [================================================================>]  Step: 66ms | Tot: 3m14s | Loss: 2.555 | Acc: 3

 [================================================================>]  Step: 22ms | Tot: 17s534ms | Loss: 2.625 | Acc: 38.014% (8363/2200 688/688 ====================>..........................................]  Step: 25ms | Tot: 6s64ms | Loss: 2.614 | Acc: 37.868% (2884/761 238/688 =========================>.......................................]  Step: 25ms | Tot: 6s836ms | Loss: 2.624 | Acc: 37.920% (3252/857 268/688 =======================================>.........................]  Step: 26ms | Tot: 10s713ms | Loss: 2.626 | Acc: 37.954% (5101/1344 420/688 ==================================================>..............]  Step: 25ms | Tot: 13s731ms | Loss: 2.625 | Acc: 38.139% (6566/1721 538/688 

Epoch: 11
 [================================================================>]  Step: 66ms | Tot: 3m13s | Loss: 2.331 | Acc: 43.236% (38048/8800 2750/2750 0 ========>.......................................................]  Step: 66ms | Tot: 29s487ms | Loss: 2.235 | Acc: 44.867% (5987/13

 [================================================================>]  Step: 66ms | Tot: 3m11s | Loss: 2.134 | Acc: 47.269% (41597/8800 2750/2750 0 ====>..........................................................]  Step: 66ms | Tot: 20s402ms | Loss: 1.985 | Acc: 50.462% (4699/931 291/2750 ===================>.............................................]  Step: 67ms | Tot: 56s604ms | Loss: 2.021 | Acc: 49.732% (12827/2579 806/2750 =================================>...............................]  Step: 69ms | Tot: 1m39s | Loss: 2.082 | Acc: 48.317% (21986/4550 1422/2750 ====================================>............................]  Step: 68ms | Tot: 1m48s | Loss: 2.087 | Acc: 48.190% (23933/4966 1552/2750 ======================================>..........................]  Step: 70ms | Tot: 1m54s | Loss: 2.091 | Acc: 48.095% (25117/5222 1632/2750 ===========================================>.....................]  Step: 68ms | Tot: 2m8s | Loss: 2.103 | Acc: 47.890% (28228/5894 1842/2

 [================================================================>]  Step: 65ms | Tot: 3m12s | Loss: 1.968 | Acc: 50.690% (44607/8800 2750/2750 0 =================>...............................................]  Step: 72ms | Tot: 52s9ms | Loss: 1.841 | Acc: 53.504% (12841/2400 750/2750 =====================>...........................................]  Step: 68ms | Tot: 1m4s | Loss: 1.867 | Acc: 52.878% (15652/2960 925/2750 =======================>.........................................]  Step: 68ms | Tot: 1m7s | Loss: 1.873 | Acc: 52.705% (16444/3120 975/2750 =================================>...............................]  Step: 66ms | Tot: 1m39s | Loss: 1.908 | Acc: 51.812% (23676/4569 1428/2750 =======================================================>.........]  Step: 72ms | Tot: 2m45s | Loss: 1.951 | Acc: 51.019% (38595/7564 2364/2750 
 [================================================================>]  Step: 23ms | Tot: 17s56ms | Loss: 2.428 | Acc: 42.486% (9347/2200 688/6

In [ ]:
del net

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = GoogLeNet(in_dim=in_dim,num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'GoogleNet') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1

In [ ]:
del net

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = ResNeXt29_2x64d(in_dim=in_dim,num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'ResNext') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1

In [ ]:
del net

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

net = DLA(in_dim=in_dim,num_classes=len(classes))
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

early_stopping = EarlyStopping(tolerance=5, min_delta=1)

epoch = 0
while(not(early_stopping.early_stop)):
    train_loss = train(epoch) / len(train_dataloader)
    validation_loss = test(epoch,'DLA') / len(test_dataloader)
    scheduler.step()
    early_stopping(train_loss,validation_loss)
    epoch+=1